In [7]:
pip install tensorflow-hub

Note: you may need to restart the kernel to use updated packages.


In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
print(tf.__version__)

2.17.0


In [6]:
model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
concrete_func = model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

@tf.function(input_signature=[tf.TensorSpec(shape=[1, 200, 200, 3], dtype=tf.float32)])
def f(input):
  return concrete_func(input);

converter = tf.lite.TFLiteConverter.from_concrete_functions([f.get_concrete_function()], model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the TF Lite model.
with tf.io.gfile.GFile('ESRGAN.tflite', 'wb') as f:
  f.write(tflite_model)

esrgan_model_path = './ESRGAN.tflite'

INFO:tensorflow:Assets written to: /tmp/tmp9a7g2pjg/assets


INFO:tensorflow:Assets written to: /tmp/tmp9a7g2pjg/assets
W0000 00:00:1724856905.117833 3581062 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1724856905.117858 3581062 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-08-28 14:55:05.118093: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp9a7g2pjg
2024-08-28 14:55:05.140315: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-08-28 14:55:05.140343: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp9a7g2pjg
2024-08-28 14:55:05.275543: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-08-28 14:55:05.861817: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp9a7g2pjg
2024-08-28 14:55:06.069805: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 951703

In [7]:
img_path = 'sr-edge/dataset/div2k/images/DIV2K_train_LR_bicubic/X4/0001x4.png'

In [8]:
lr = tf.io.read_file(img_path)
lr = tf.image.decode_jpeg(lr)
lr = tf.expand_dims(lr, axis=0)
lr = tf.cast(lr, tf.float32)

new_size = [200, 200]
lr = tf.image.resize(lr, new_size)

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=esrgan_model_path)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Run the model
interpreter.set_tensor(input_details[0]['index'], lr)
interpreter.invoke()

# Extract the output and postprocess it
output_data = interpreter.get_tensor(output_details[0]['index'])
sr = tf.squeeze(output_data, axis=0)
sr = tf.clip_by_value(sr, 0, 255)
sr = tf.round(sr)
sr = tf.cast(sr, tf.uint8)

2024-08-28 14:55:08.102375: W tensorflow/compiler/mlir/tools/kernel_gen/tf_gpu_runtime_wrappers.cc:40] 'cuModuleLoadData(&module, data)' failed with 'CUDA_ERROR_UNSUPPORTED_PTX_VERSION'

2024-08-28 14:55:08.102408: W tensorflow/compiler/mlir/tools/kernel_gen/tf_gpu_runtime_wrappers.cc:40] 'cuModuleGetFunction(&function, module, kernel_name)' failed with 'CUDA_ERROR_INVALID_HANDLE'

2024-08-28 14:55:08.102419: W tensorflow/core/framework/op_kernel.cc:1828] INTERNAL: 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE'
2024-08-28 14:55:08.102433: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INTERNAL: 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE'


InternalError: {{function_node __wrapped__Cast_device_/job:localhost/replica:0/task:0/device:GPU:0}} 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE' [Op:Cast] name: 

In [ ]:
lr = tf.cast(tf.squeeze(lr, axis=0), tf.uint8)
plt.figure(figsize = (1, 1))
plt.title('LR')
plt.imshow(lr.numpy());

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)        
plt.title(f'ESRGAN (x4)')
plt.imshow(sr.numpy());

bicubic = tf.image.resize(lr, [200, 200], tf.image.ResizeMethod.BICUBIC)
bicubic = tf.cast(bicubic, tf.uint8)
plt.subplot(1, 2, 2)   
plt.title('Bicubic')
plt.imshow(bicubic.numpy());

In [ ]:
plt.imshow(sr.numpy())

In [ ]:
lower_red = np.array([0, 120, 70])
upper_red = np.array([10, 255, 255])

In [ ]:
sr_np = sr.numpy() 
sr_bgr = cv2.cvtColor(sr_np, cv2.COLOR_RGB2BGR)

In [ ]:
def detect_color(frame):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask_red = cv2.inRange(hsv_frame, lower_red, upper_red)
    
    contours, _ = cv2.findContours(mask_red, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 500: 
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, "Warning: Red Object Detected!", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            print("Warning: Red Object Detected!")

    return frame

In [ ]:
result_frame = detect_color(sr_bgr)

In [ ]:
result_rgb = cv2.cvtColor(result_frame, cv2.COLOR_BGR2RGB)
pil_img = Image.fromarray(result_rgb)

In [ ]:
plt.figure(figsize=(8, 8))
plt.title('Processed SR Image with Color Detection')
plt.imshow(pil_img)
plt.axis('off')
plt.show()

## Model Conversion.ipynb(TFLite, Quantize to int8, Compile for Edge TPU, TFLite float32)
# https://github.com/PIC4SeR/EdgeSRGAN/blob/master/Model_Conversion.ipynb